In [0]:
import pandas as pd
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
import keras.utils
from sklearn.metrics import accuracy_score

In [0]:
drive.mount('/content/drive')

In [0]:
loc = '/content/drive/My Drive/Kaggle - Digit Recognizer/Data/'

In [0]:
train = pd.read_csv(loc+'train.csv')
test = pd.read_csv(loc+'test.csv')

In [0]:
d_tr = {}
img = []
lab = []
for i in range(len(train)):
  lab.append(train['label'][i])
  l = []
  for j in range(784):
    l.append(train['pixel'+str(j)][i])
  m = []
  for j in range(0, 784, 28):
    m.append(l[j:j+28])
  m_np = np.array(m)
  img.append(m_np)
d_tr['images'] = img
d_tr['label'] = lab

In [0]:
d_ts = {}
img = []
for i in range(len(test)):
  l = []
  for j in range(784):
    l.append(test['pixel'+str(j)][i])
  m = []
  for j in range(0, 784, 28):
    m.append(l[j:j+28])
  m_np = np.array(m)
  img.append(m_np)
d_ts['images'] = img

In [0]:
tr = {}
tr['images'] = d_tr['images'][0:int(0.8*len(train))]
tr['labels'] = d_tr['label'][0:int(0.8*len(train))]
dev = {}
dev['images'] = d_tr['images'][int(0.8*len(train)):int(0.9*len(train))]
dev['labels'] = d_tr['label'][int(0.8*len(train)):int(0.9*len(train))]
ts = {}
ts['images'] = d_tr['images'][int(0.9*len(train)):]
ts['labels'] = d_tr['label'][int(0.9*len(train)):]

In [0]:
model = Sequential()
model.add(Conv2D(6, kernel_size=5, padding='valid', strides=1, activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(Conv2D(6, kernel_size=5, padding='valid', strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
tr_x = np.array(tr['images'])
tr_x = tr_x.reshape(len(tr['images']), 28, 28, 1)
dev_x = np.array(dev['images'])
dev_x = dev_x.reshape(len(dev['images']), 28, 28, 1)
ts_x = np.array(ts['images'])
ts_x = ts_x.reshape(len(ts['images']), 28, 28, 1)

In [0]:
tr_y = keras.utils.to_categorical(tr['labels'])
dev_y = keras.utils.to_categorical(dev['labels'])
ts_y = keras.utils.to_categorical(ts['labels'])

In [0]:
model.fit(tr_x, tr_y, validation_data=(dev_x, dev_y), epochs=20)

In [0]:
ts_p = model.predict(ts_x)
ts_p_m = [np.argmax(ts_p[i]) for i in range(len(ts_p))]
print('Accuracy: ', accuracy_score(ts['labels'], ts_p_m))

In [0]:
test_x = np.array(d_ts['images'])
test_x = test_x.reshape(28000, 28, 28, 1)

In [0]:
test_p = model.predict(test_x)
d = {}
img_id = []
label = []
for i in range(len(test)):
  img_id.append(i+1)
  label.append(np.argmax(test_p[i]))
d['ImageId'] = img_id
d['Label'] = label
df = pd.DataFrame(d)
df.to_csv(loc+'result.csv', index=False)

In [0]:
df